In [1]:
import sys
sys.path.append("/home/ms10596/PycharmProjects/match")
from utils.one_stop_english import load_advanced_elementary

In [2]:
advanced, elementary = load_advanced_elementary()

In [3]:
from statistics import mean
print("mean len of adv:", mean([len(i) for i in advanced]))
print("mean len of elem:", mean([len(i) for i in elementary]))

mean len of adv: 126.8915050784857
mean len of elem: 111.70452446906741


In [4]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [7]:
import io
import json
tokenizer = Tokenizer()
tokenizer.fit_on_texts(advanced+elementary)
tokenizer_json = tokenizer.to_json()
with io.open('simplification_tokenizer.json', 'w', encoding='utf-8') as f:
    f.write(json.dumps(tokenizer_json, ensure_ascii=False))
len(tokenizer.word_index)

5760

In [20]:
source_sequences = tokenizer.texts_to_sequences(advanced)
import numpy as np
print(np.shape(source_sequences))
print(source_sequences[0])
print(advanced[0])

(2166,)
[1, 3290, 475, 197, 22, 1089, 9, 34, 948, 4, 28, 5, 118, 233, 715, 412, 1430, 517, 17, 1, 164, 302, 1876, 1090, 41, 64, 1297, 1, 96, 3, 41, 277, 829, 9, 884, 1877, 1, 3291, 3, 1091, 1878, 2, 71, 204, 885, 606]
the seattle-based company has applied for its brand to be a top-level domain name (currently .com), but the south american governments argue this would prevent the use of this internet address for environmental protection, the promotion of indigenous rights and other public interest uses.


In [21]:
target_sequences = tokenizer.texts_to_sequences(elementary)
print(np.shape(target_sequences))
print(target_sequences[0])
print(elementary[0])

(2166,)
[887, 22, 350, 9, 34, 197, 412, 4, 28, 5, 118, 233, 715, 412, 1430, 517, 17, 1, 164, 302, 1876, 212, 41, 64, 652, 1, 96, 3, 41, 277, 829, 9, 884, 1877, 1091, 1878, 2, 71, 204, 885, 606]
amazon has asked for its company name to be a top-level domain name (currently .com), but the south american governments say this would stop the use of this internet address for environmental protection, indigenous rights and other public interest uses.


In [22]:
print("mean len of adv:", max([len(i) for i in source_sequences]))
print("mean len of elem:", max([len(i) for i in target_sequences]))

mean len of adv: 60
mean len of elem: 52


In [23]:
from tensorflow.keras.utils import to_categorical
padded_target_sequences = pad_sequences(target_sequences, maxlen=60, padding='post', truncating='post')
padded_source_sequences = pad_sequences(source_sequences, maxlen=60, padding='post', truncating='post')
padded_target_sequences = to_categorical(padded_target_sequences)
print(np.shape(padded_target_sequences))
print(np.shape(padded_source_sequences))

(2166, 60, 5761)
(2166, 60)


In [24]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, TimeDistributed, RepeatVector, Dense
from tensorflow.python.keras.layers.embeddings import Embedding
from tensorflow import keras
es = keras.callbacks.EarlyStopping(monitor='val_loss',patience=10, verbose=0, mode='auto')

n_units = 256
model = Sequential()
model.add(Embedding(5761, n_units, input_length=60, mask_zero=True))
# model.add(LSTM(n_units))
# model.add(RepeatVector(tar_timesteps))
model.add(LSTM(n_units, return_sequences=True))
model.add(TimeDistributed(Dense(5761, activation='softmax')))
model.compile(optimizer='adam', loss='categorical_crossentropy')
model.fit(padded_source_sequences, padded_target_sequences, epochs=100, batch_size=64, verbose=2, validation_split=0.2,callbacks=[es])

/home/ms10596/anaconda3/envs/tf/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:112: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 1732 samples, validate on 434 samples
Epoch 1/100
 - 28s - loss: 7.7718 - val_loss: 6.5864
Epoch 2/100
 - 9s - loss: 6.6318 - val_loss: 6.3661
Epoch 3/100
 - 9s - loss: 6.4832 - val_loss: 6.2783
Epoch 4/100
 - 9s - loss: 6.4152 - val_loss: 6.2202
Epoch 5/100
 - 9s - loss: 6.3301 - val_loss: 6.1219
Epoch 6/100
 - 9s - loss: 6.2406 - val_loss: 6.0525
Epoch 7/100
 - 9s - loss: 6.1631 - val_loss: 5.9741
Epoch 8/100
 - 9s - loss: 6.0842 - val_loss: 5.9179
Epoch 9/100
 - 9s - loss: 6.0014 - val_loss: 5.8180
Epoch 10/100
 - 9s - loss: 5.9290 - val_loss: 5.7659
Epoch 11/100
 - 9s - loss: 5.8569 - val_loss: 5.6825
Epoch 12/100
 - 9s - loss: 5.7809 - val_loss: 5.6268
Epoch 13/100
 - 9s - loss: 5.7138 - val_loss: 5.5394
Epoch 14/100
 - 9s - loss: 5.6398 - val_loss: 5.4676
Epoch 15/100
 - 9s - loss: 5.5681 - val_loss: 5.4194
Epoch 16/100
 - 9s - loss: 5.5042 - val_loss: 5.3258
Epoch 17/100
 - 9s - loss: 5.4300 - val_loss: 5.2593
Epoch 18/100
 - 9s - loss: 5.3616 - val_loss: 5.1926
Epoch 1

In [25]:
id_to_word = {i:word for word,i in tokenizer.word_index.items()}
id_to_word[0] = "<space>"

In [26]:
for i in model.predict(padded_source_sequences[50:51])[0]:
#     print(np.shape(i))
    val = np.argmax(i)
    if val < 5761:
        print(id_to_word[val])

china
is
the
biggest
coal
importer
and
indonesia
is
the
biggest
coal
exporter
<space>
<space>
<space>
<space>
<space>
<space>
<space>
<space>
<space>
<space>
<space>
<space>
<space>
<space>
<space>
<space>
<space>
<space>
<space>
<space>
<space>
<space>
<space>
<space>
<space>
<space>
<space>
<space>
<space>
<space>
<space>
<space>
<space>
<space>
<space>
<space>
<space>
<space>
<space>
<space>
<space>
<space>
<space>
<space>
<space>
<space>
<space>


In [27]:
model.save('sha2y.h5')

In [ ]:
def simplify(text):
    